In [ ]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from itertools import cycle
import numpy as np

data_directory = './'

In [ ]:
image_path = Path(data_directory)
image_path.mkdir(parents=True, exist_ok=True)

images_to_generate = {
    'BIAS': 5,
    'DARK': 10,
    'FLAT': 3,
    'LIGHT': 10
}

exposure_times = {
    'BIAS': [0.0],
    'DARK': [5.0, 30.0],
    'FLAT': [5.0, 6.1, 7.3],
    'LIGHT': [30.0],
}

filters = {
    'FLAT': 'V',
    'LIGHT': 'V'
}

objects = {
    'LIGHT': ['m82', 'xx cyg']
}

image_size = [300, 200]

In [ ]:
image_number = 0
for image_type, num in images_to_generate.items():
    exposures = cycle(exposure_times[image_type])
    try:
        filts = cycle(filters[image_type])
    except KeyError:
        filts = []
    
    try:
        objs = cycle(objects[image_type])
    except KeyError:
        objs = []
    for _ in range(num):
        img = CCDData(data=np.random.randn(*image_size), unit='adu')
        img.meta['IMAGETYP'] = image_type
        img.meta['EXPOSURE'] = next(exposures)
        if filts:
            img.meta['FILTER'] = next(filts)
        if objs:
            img.meta['OBJECT'] = next(objs)
        image_name = str(image_path / f'img-{image_number:04d}.fits')
        img.write(image_name)
        print(image_name)
        image_number += 1

In [ ]:
image_name = 'img-0001.fits'

image_path = Path(data_directory) / image_name

hdu_list = fits.open(image_path)
hdu_list.info()

In [ ]:
hdu = hdu_list[0]
hdu.header

In [ ]:
hdu.data

In [ ]:
ccd = CCDData.read(image_path)

In [ ]:
ccd.header

In [ ]:
ccd.data

In [ ]:
from ccdproc import ImageFileCollection
im_collection = ImageFileCollection(data_directory)

In [ ]:
im_collection.summary

In [ ]:
for a_flat, fname in im_collection.hdus(imagetyp='LIGHT', object='m82', return_fname=True):
    print(f'In file {fname} the exposure is:', a_flat.header['EXPOSURE'], 'with standard deviation ', a_flat.data.std())

In [ ]:
for a_flat, fname in im_collection.ccds(bunit='ADU', return_fname=True):
    print(a_flat.unit)

In [ ]:
a_flat.header

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('guide.mplstyle')

from astropy.nddata import CCDData
from astropy.visualization import hist

import numpy as np

from convenience_functions import show_image

In [ ]:
one_bias = CCDData.read('single_bias_thermoelectric.fit.bz2', unit='adu')
one_hundred_bias = CCDData.read('combined_bias_100_images.fit.bz2', unit='adu')

In [ ]:
fig, (ax_1_bias, ax_avg_bias) = plt.subplots(1, 2, figsize=(30, 15))

show_image(one_bias.data, cmap='gray', ax=ax_1_bias, fig=fig, input_ratio=8)
ax_1_bias.set_title('Single bias image')
show_image(one_hundred_bias.data, cmap='gray', ax=ax_avg_bias, fig=fig, input_ratio=8)
ax_avg_bias.set_title('100 bias images combined');

In [ ]:
plt.figure(figsize=(20,10))
hist(one_bias.data[:, 15:-30].flatten(), bins=800, alpha=0.4, label='One bias', color='deepskyblue')
hist(np.concatenate((one_bias.data[:, :15].flatten(), one_bias.data[:, -30:].flatten())), bins=400, alpha=0.2, label='One bias (edges only)', color='lightskyblue')
#hist(, bins=800, alpha=0.2, label='One bias (edges only)', color='darkblue')
hist(one_hundred_bias.data[:, 15:-30].flatten(), bins=800, alpha=0.4, label='One hundred bias images', color='darkgreen')
hist(np.concatenate((one_hundred_bias.data[:, :15].flatten(), one_hundred_bias.data[:, -30:].flatten())), bins=800, alpha=0.4, label='One hundred bias images (edges only)', color='lightgreen')

#hist(one_hundred_bias.data[:, :15].flatten(), bins=800, alpha=0.4, label='One hundred bias images', color='darkgreen')

plt.grid()
plt.xlim(975, 1400)
plt.legend()
plt.xlabel('Pixel value')
plt.ylabel('Number of pixels')
plt.semilogy();